In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

import pandas as pd
import numpy as np

In [33]:
Dataset = pd.read_csv('balanced_filled_Dataset-vf.csv')

In [34]:
Dataset.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,Y
0,2892.0,75.000000,7.0,95.0,9.0,1889.0,228.0,228.0,133.0,2371.0,0.79,39.0,3,42,Apple
1,3208.0,0.000000,9.0,124.0,-2.0,5394.0,206.0,222.0,154.0,900.0,0.97,-740.0,1,42,Apple
2,3245.0,97.630426,3.0,564.0,66.0,4387.0,220.0,233.0,150.0,2650.0,0.55,4.0,1,42,Apple
3,3157.0,307.000000,27.0,120.0,35.0,2971.0,138.0,213.0,211.0,2467.0,0.19,637.0,1,44,Apple
4,3246.0,18.000000,9.0,120.0,11.0,4333.0,213.0,221.0,144.0,972.0,0.93,833.0,1,59,Apple


In [35]:
X = Dataset.drop('Y', axis=1)
Y = Dataset['Y']

In [36]:
X.shape, Y.shape

((7774, 14), (7774,))

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=20211008, stratify=Y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8, random_state=20211008, 
                                                  stratify=y_train)

In [38]:
import itertools
criterion = ["gini", "entropy"] # The function to measure the quality of a split. 
splitter = ["best", "random"] # The strategy used to choose the split at each node.
max_depth = [None, 100, 1000] # The maximum depth of the tree.
max_features = [None, "auto", "sqrt", "log2"] # The number of features to consider when looking for the best split   
parameters = [criterion,splitter,max_depth, max_features]  
parameters_combinations = list(itertools.product(*parameters))
len(parameters_combinations) 


48

In [39]:
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from imblearn.metrics import geometric_mean_score

In [40]:
from sklearn.ensemble import AdaBoostClassifier
# training and hyper-parameter tuning

####### ADD YOUR CODE ####
n_estimators_array = [10,100]
best_acc_params = {"base_estimator": None, "n_estimators": None,"accuracy": 0}


best_acc_params = {"accuracy": {
                             "average_score": 0,
                             "f1_score_macro": 0,
                             "f1_score_micro": 0,
                             "MCC": 0,
                             "Gmean": 0
                         }}
print('Decision Tree Classifier: ')
for params in parameters_combinations:
    
    des_tree_clf = DecisionTreeClassifier(criterion=params[0], splitter=params[1], 
                                          max_depth=params[2], max_features=params[3])
    for n_estimators in n_estimators_array:
        print(f'n_estimators: {n_estimators}')
        clf = AdaBoostClassifier(base_estimator=des_tree_clf,n_estimators=n_estimators)
        clf = clf.fit(X_train, y_train)
        y_predict = clf.predict(X_val)

        f1_score_macro = f1_score(y_val, y_predict, average='macro')
        f1_score_micro = f1_score(y_val, y_predict, average='micro')
        MCC_score = matthews_corrcoef(y_val, y_predict)
        Gmean_score = geometric_mean_score(y_val,y_predict, average='macro')
        accuracy = (f1_score_macro + f1_score_micro + MCC_score + Gmean_score) / 4.0
        
        if accuracy > 0.50:
            print(f'parameters: \n criterion: {params[0]}, splitter: {params[1]}, max depth: {params[2]}, max features: {params[3]}')
            print(f'accuracy: {accuracy}')
            if accuracy > best_acc_params['accuracy']['average_score']:
                best_acc_params.update({"base_estimator": {"name": "Decision Tree", "criterion": params[0], "splitter": params[1], "max_depth": params[2],
                         "max_features": params[3]}, "n_estimators": n_estimators,"accuracy": {
                             "average_score": accuracy,
                             "f1_score_macro": f1_score_macro,
                             "f1_score_micro": f1_score_micro,
                             "MCC": MCC_score,
                             "Gmean": Gmean_score
                         }})
    print('--------------------------------------------------------------')
    
lr = LogisticRegression();
bnb = BernoulliNB()
gnb = GaussianNB()
for base_est in [lr, bnb, gnb]:
    for n_estimators in n_estimators_array:
            print(f'n_estimators: {n_estimators}')
            clf = AdaBoostClassifier(base_estimator=base_est,n_estimators=n_estimators)
            clf = clf.fit(X_train, y_train)
            y_predicts = clf.predict(X_val)

            f1_score_macro = f1_score(y_val, y_predicts, average='macro')
            f1_score_micro = f1_score(y_val, y_predicts, average='micro')
            MCC_score = matthews_corrcoef(y_val, y_predicts)
            Gmean_score = geometric_mean_score(y_val,y_predicts, average='macro')
            accuracy = (f1_score_macro + f1_score_micro + MCC_score + Gmean_score) / 4.0

            if accuracy > 0.50:
                print(f'parameters: \n criterion: {params[0]}, splitter: {params[1]}, max depth: {params[2]}, max features: {params[3]}')
                print(f'accuracy: {accuracy}')
                if accuracy > best_acc_params['accuracy']['average_score']:
                    best_acc_params.update({"base_estimator": {"name": "Decision Tree", "criterion": params[0], "splitter": params[1], "max_depth": params[2],
                             "max_features": params[3]}, "n_estimators": n_estimators,"accuracy": {
                                 "average_score": accuracy,
                                 "f1_score_macro": f1_score_macro,
                                 "f1_score_micro": f1_score_micro,
                                 "MCC": MCC_score,
                                 "Gmean": Gmean_score
                             }})


Decision Tree Classifier: 
n_estimators: 10
parameters: 
 criterion: gini, splitter: best, max depth: None, max features: None
accuracy: 0.8956807495519871
n_estimators: 100
parameters: 
 criterion: gini, splitter: best, max depth: None, max features: None
accuracy: 0.8926627564547064
--------------------------------------------------------------
n_estimators: 10


C:\Users\ubi-A\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\ubi-A\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\ubi-A\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\ubi-A\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\ubi-A\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\ubi-A\anaconda3\lib\site-p

parameters: 
 criterion: gini, splitter: best, max depth: None, max features: auto
accuracy: 0.8681698657479874
n_estimators: 100
parameters: 
 criterion: gini, splitter: best, max depth: None, max features: auto
accuracy: 0.8999018236932148
--------------------------------------------------------------
n_estimators: 10
parameters: 
 criterion: gini, splitter: best, max depth: None, max features: sqrt
accuracy: 0.885163921818588
n_estimators: 100
parameters: 
 criterion: gini, splitter: best, max depth: None, max features: sqrt
accuracy: 0.8632298397967569
--------------------------------------------------------------
n_estimators: 10
parameters: 
 criterion: gini, splitter: best, max depth: None, max features: log2
accuracy: 0.8986922875460058
n_estimators: 100
parameters: 
 criterion: gini, splitter: best, max depth: None, max features: log2
accuracy: 0.8571382848773408
--------------------------------------------------------------
n_estimators: 10


C:\Users\ubi-A\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\ubi-A\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\ubi-A\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


parameters: 
 criterion: gini, splitter: best, max depth: 100, max features: None
accuracy: 0.8897212565146622
n_estimators: 100
parameters: 
 criterion: gini, splitter: best, max depth: 100, max features: None
accuracy: 0.8927238357651393
--------------------------------------------------------------
n_estimators: 10
parameters: 
 criterion: gini, splitter: best, max depth: 100, max features: auto
accuracy: 0.8551886258920136
n_estimators: 100
parameters: 
 criterion: gini, splitter: best, max depth: 100, max features: auto
accuracy: 0.8651766171483448
--------------------------------------------------------------
n_estimators: 10


...

parameters: 
 criterion: entropy, splitter: random, max depth: 1000, max features: log2
accuracy: 0.520068328674256
n_estimators: 10
n_estimators: 100


C:\Users\ubi-A\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


n_estimators: 10
parameters: 
 criterion: entropy, splitter: random, max depth: 1000, max features: log2
accuracy: 0.5283856951381982
n_estimators: 100


C:\Users\ubi-A\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\ubi-A\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


parameters: 
 criterion: entropy, splitter: random, max depth: 1000, max features: log2
accuracy: 0.5606156214997595


In [41]:
best_acc_params

{'accuracy': {'average_score': 0.92542905618562,
  'f1_score_macro': 0.9231627729769611,
  'f1_score_micro': 0.9188102893890675,
  'MCC': 0.9049610278879228,
  'Gmean': 0.9547821344885288},
 'base_estimator': {'name': 'Decision Tree',
  'criterion': 'entropy',
  'splitter': 'best',
  'max_depth': 1000,
  'max_features': None},
 'n_estimators': 10}

In [42]:
base_estimator_params = best_acc_params['base_estimator']
base_estimator = DecisionTreeClassifier(criterion=base_estimator_params['criterion'], 
                                                                         splitter=base_estimator_params['splitter'], 
                                                                         max_depth=base_estimator_params['max_depth'],
                                                                         max_features=base_estimator_params['max_features'])
clf = AdaBoostClassifier(base_estimator=base_estimator,n_estimators=best_acc_params['n_estimators'])
clf = clf.fit(X_train, y_train)
y_predicts = clf.predict(X_test)




f1_score_macro = f1_score(y_test, y_predicts, average='macro')
f1_score_micro = f1_score(y_test, y_predicts, average='micro')
MCC_score = matthews_corrcoef(y_test, y_predicts)
Gmean_score = geometric_mean_score(y_test,y_predicts, average='macro')


print(f'f1_score (macro): {f1_score_macro}')
print(f'f1_score (micro): {f1_score_micro}')
print(f'MCC: {MCC_score}')
print(f'Gmean: {Gmean_score}')

f1_score (macro): 0.9185644667498808
f1_score (micro): 0.9144694533762058
MCC: 0.8998424828159912
Gmean: 0.9516340657758438


C:\Users\ubi-A\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
